## LSI Search for related documents based on TF-IDF

[Benefit of LSI](https://en.wikipedia.org/wiki/Latent_semantic_analysis#Benefits_of_LSI): LSI overcomes two of the most problematic constraints of Boolean keyword queries: multiple words that have similar meanings (synonymy) and words that have more than one meaning (polysemy)[1].

First, we define the similarity score by cosine similarity, then we construct the query vector based on the tf_idf matrix, later by reducing the dimension, we get a k dimensional concept space. From that concept space, we can calculate a relevance scores for the query and each document, thus return the most relevant documents. 

**Reference:**

[1] Wikipedia contributors. (2018, September 1). Latent semantic analysis. In Wikipedia, The Free Encyclopedia. Retrieved 10:48, September 20, 2018, from https://en.wikipedia.org/w/index.php?title=Latent_semantic_analysis&oldid=857577135

#### 1. Run 04-TF-IDF_Raw_Implementation.ipynb first, to make sure tf-idf implementation can be invoked

In [1]:
%run 04-TF-IDF_Raw_Implementation.ipynb

........................................................................................................................................................................................................



04-TF-IDF_Raw_Implementation.ipynb:53: RuntimeWarning: divide by zero encountered in true_divide
  "    word2index = {}\n",
04-TF-IDF_Raw_Implementation.ipynb:53: RuntimeWarning: invalid value encountered in true_divide
  "    word2index = {}\n",


CPU times: user 2.66 s, sys: 299 ms, total: 2.96 s
Wall time: 2.32 s


In [2]:
import numpy as np
from numpy import linalg

from scipy import spatial

import matplotlib.pyplot as plt
from matplotlib import colors

In [3]:
# defining the similarity function
def similarity(u, v):
    return 1 - spatial.distance.cosine(u, v)

#### Find most related documents according to a list of keywords
- params: 
    - doc_sim: dict, key:index of the document, value:similarity of this specific document to those keywords
    - index2document: dict, key:document name (str), value:index (int)
- variables: 
    - related_doc: dict, key:document name, value: similarity of this document to those keywords

In [4]:
def get_doc_relevance(doc_sim, index2document):
    related_doc = {}
    for doc, sim in sorted(doc_sim.items(), key=lambda x: x[1], reverse=True):
        related_doc[index2document[doc]] = sim
    return related_doc

#### 2. Run LSI
Latent semantic analysis  is a technique in natural language processing, in particular distributional semantics, of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. 

*prerequisite*: run [TF-IDF](04-TF-IDF_Raw_Implementation.ipynb), in order to invoke tf_idf_prerocessed_doc function!

- params: 
    - q: query (str)
    - k: k-concept space
    - tf_idf: matrix obtained after invoking function: tf_idf_prerocessed_doc(processed_doc_path, 200)
    - word2index: dict, obtained after invoking function: tf_idf_prerocessed_doc(processed_doc_path, 200)
    - index2documtn: dict, obtained after invoking function: tf_idf_prerocessed_doc(processed_doc_path, 200)
- variables: 
    - u, s, v: result of Singular value decomposition on tf-idf matrix
    - q_k: query vector -- q is mapped to the k-concept space,
        
        $q_k=q^{T}U_k\Sigma^{-1}_{k}$ --> $q_k = q.T.dot(u).dot(s.inv())$
        
    - doc_similarity: dict, key:index of the document, value:similarity of this specific document to those keywords

In [5]:
def latent_semantic_indexing(q, tf_idf, k, word2index, index2document):

    # preprocssing the query vector
    query = []
    doc_similarity = {}

    q_v = np.zeros(len(word2index))
    for w in q.split():
        try:
            q_v[word2index[w]] += 1
            query.append(word2index[w])
        except:
            pass
#             print('keywords not found')

    u, s, v = linalg.svd(tf_idf)
    u = u[:, :k]
    s_ = np.zeros((k, k))

    for i in range(k):
        s_[i, i] = s[i]
    v = v[:k, :]
    
    q_v = q_v.reshape(1, -1)
    q_v = np.matmul(q_v, u)
    s_ = linalg.inv(s_)
    q_v = np.matmul(q_v, s_)

    for i in range(v.shape[1]):
        sim = similarity(q_v, v[:, i])
        doc_similarity[i] = sim
    related_doc = get_doc_relevance(doc_similarity, index2document)

    return related_doc

In [8]:
%%time
# please modify the relative path
# processed_doc_path = '/home/bit/ma0/LabShare/data/chui_ma/spacy_corpus/'
processed_doc_path = '../spacy_corpus/'
tf_idf, word2index, index2document, inv_doc_freq = tf_idf_prerocessed_doc(processed_doc_path, 20)

....................

CPU times: user 214 ms, sys: 11.7 ms, total: 226 ms
Wall time: 171 ms


In [9]:
%%time
# get related documents with relevance score
q = 'auto autobahn frankfurt'
related_doc = latent_semantic_indexing(q, tf_idf, 10, word2index, index2document)

CPU times: user 24.4 s, sys: 2.17 s, total: 26.5 s
Wall time: 7.56 s


#### Wrap the result with pandas DataFrame, and Coloring the table

In [10]:
def visualize_LSI_results(related_doc):

    def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
        rng = M - m
        norm = colors.Normalize(m - (rng * low),
                                M + (rng * high))
        normed = norm(s.values)
        c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
        return ['background-color: %s' % color for color in c]

    df = pd.DataFrame(related_doc, index=['relevance score'])
    df = df.transpose()
    return df.style.apply(background_gradient,
                          cmap='PuBu',
                          m=df.min().min(),
                          M=df.max().max(),
                          low=0,
                          high=0.2)

In [11]:
visualize_LSI_results(related_doc)

,relevance score
DIC-Asset-AnnualReport-2014,0.971754
DIC-Asset-AnnualReport-2013,0.971081
Fraport-QuarterlyReport-2016-Q1,0.93491
DeutscheWohnen-AnnualReport-2016,0.910891
Eventim-AnnualReport-2014,0.851511
DeutscheBank-QuarterlyReport-2013-Q3,0.776819
Bayer-QuarterlyReport-2014-Q3,0.755609
BASF-QuarterlyReport-2017-Q1,0.752401
Daimler-QuarterlyReport-2014-Q2,0.72748
Deutz-QuarterlyReport-2013-Q2,0.569965


Using LSI search, we may found out those documents related to our keywords, however, if we need to verify the topics of a document, we need to refer to [LDA](07-LDA.ipynb)